# Ćwiczenia 4 - Regularyzacja

## Early stopping
Polega na zatrzymaniu uczenia w momencie kiedy wybrana metryka przestała się poprawiać dla danych walidacyjnych.
Keras pozwala na zaimplementowac tę metodę jako callback, który można podać jako parametr w metodzie `fit`, np.:

```python
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=2, restore_best_weights=True, verbose=1)

model.fit(..., callbacks=[early_stopping])
```

## Regularyzacja L1/L2

Do funkcji błędu dodawany jest dodatkowy termin penalizujący zbyt duże wagi:

$$ \tilde{J}(W) = J(W) + \lambda \sum_i w_i^2 $$
$$ \tilde{J}(W) = J(W) + \lambda \sum_i |w_i| $$

Aby zastosować tę metodę dla warstwy, należy ustawić parametr `kernel_regularizer`, np:

```python
tf.keras.layers.Dense(..., kernel_regularizer=tf.keras.regularizers.l2(0.1))
```

## Regularyzacja Dropout

Regularyzacja Dropout polega na losowym wyłączaniu pewnej części neuronów w sieci podczas uczenia.
Metodę tę można zaimplementowac dodając warstwę `Dropout` bezpośrednio po regularyzowanej warstwie.

## Augmentacja danych

Polega na wprowadzaniu drobnych zmian w danych treningowych. 
Technika ta jest szczególnie przydatna w przetwarzaniu obrazów. Różne transformacje można zastosować dodając odpowiednie warstwy na wejściu sieci, np.: `RandomRotation`, `RandomZoom`, `RandomBrightness`. Pełna lista dostępncyh transformacji: https://keras.io/api/layers/preprocessing_layers/image_augmentation/

## Zadania
1. Wczytaj zbiór `imdb` z Ćwiczeń 03. Zastosuj sieć wielowarstwową do klasyfikacji zbioru (możesz wykorzystać sieć z przykładu). Stwórz wykres pokazujący błąd dla danych treningowych/walidacyjnych w kolejnych epokach i zaobserwuj efekt przetrenowania. Następnie uruchom sieć ponownie, stosując Early Stopping aby zatrzymać uczenie w momencie, kiedy funkcja straty przestanie się zminiejszać dla danych walidacyjnych.
2. Dodaj do warstw ukrytych sieci z Zadania 1. regularyzację L1/L2. Dobierz odpowiednie wartości współczynnika regularyzacji. Trenuj sieci bez stosowania Early Stopping. Stwórz wykresy krzywych uczenia i porównaj działanie regularyzowanych sieci z siecią z Zadania 1.
3. Dodaj do warstw ukrytych sieci z Zadania 1. regularyzację Dropout. Dobierz odpowiedni parametr `rate`. Stwórz wykres krzywej uczenia i porównaj działanie sieci stosującej Dropout z siecią z Zadania 1.
4. (opcjonalne) Dodaj do sieci klasyfikującej zbiór CIFAR-10 stworzonej na wcześniejszych zajęciach augmentację obrazów. Bezpośrednio po warstwie `Input` i przed warstwą `Flatten` dodaj wybrane transformacje.